In [1]:
!pip install scanpy
# !pip install openai

# Libraries importation
from __future__ import annotations
import os # to import secret variables saved on the pc
import re # to isolate and clean the json objects from the model's response
import json # to convert json objects in python data structures
from dataclasses import dataclass # to define compact data classes
from typing import List, Dict, Any, Tuple, Optional # to annotate data types
from openai import OpenAI

# API configuration (Colab)
from google.colab import userdata  # type: ignore
_api_key = userdata.get('API-KEY')
client = OpenAI(api_key=_api_key)
MODEL = "gpt-4o-mini"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.6 MB/s eta 0:00:00


In [2]:
# --- Utilities ---------------------------------------------------------------
def extract_json(content: str) -> Tuple[Optional[Any], Optional[Exception]]:
    """Robustly parse JSON from LLM outputs.
    - Strips code fences
    - Attempts direct json.loads
    - Fallback: regex to first {...} or [...]
    Returns (obj, err)
    """
    if not content:
        return None, ValueError("Empty content")

    # Remove single or triple backticks fences, optionally labeled as json
    text = content.strip()
    text = re.sub(r"^```(?:json)?\s*|\s*```$", "", text, flags=re.IGNORECASE | re.MULTILINE)

    # Direct attempt
    try:
        return json.loads(text), None
    except Exception:
        pass

    # Find first JSON-looking block
    m = re.search(r"(\{.*\}|\[.*\])", text, flags=re.DOTALL)
    if m:
        try:
            return json.loads(m.group(1)), None
        except Exception as e:
            return None, e

    return None, ValueError("No JSON block found")

In [3]:
# --- Data structures ---------------------------------------------------------

@dataclass
class Step:
    step: int
    title: str
    description: str

@dataclass
class Result:
    step: int
    title: str
    description: str
    code: str
    justification: str

In [10]:
# --- Prompts -----------------------------------------------------------------

def make_planner_prompt(user_task: str, dataset_description: str) -> str:
    # NOTE: double braces in JSON example (f-string)
    return f"""
Act as an expert in single-cell RNA-seq analysis. The user will provide a task.
Your job: decompose it into an ordered list of steps (max 8). Each step must have a title and a short description.
Be concrete and actionable. Prefer Scanpy + CellTypist in Python. Avoid R-only tools.

Avoid duplicating normalization or HVG selection steps — include them once.

If clusters (e.g., `leiden`) exist, suggest using them for annotation (e.g., CellTypist with group_by='leiden').
Add a step for reviewing or verifying AnnData structure if appropriate.

Dataset description: {dataset_description}
User task: {user_task}

Return ONLY a valid JSON array (no prose). Example format:
[
  {{"step": 1, "title": "QC & normalization", "description": "Filter low-quality cells; normalize counts; log1p."}},
  {{"step": 2, "title": "HVG + PCA", "description": "Find highly variable genes; run PCA."}}
]
"""


def make_executor_prompt(step_description: str, available_tools: List[str], context_code: str) -> str:
    # Filter out R-only tools
    r_only = {"ScType", "Harmony", "Combat", "Slingshot"}
    py_tools = [t for t in available_tools if t not in r_only]

    tool_docs = {
        "Scanpy": (
            "Python library for scRNA-seq: preprocessing, HVGs, PCA/UMAP, clustering, visualization.\n"
            "Use `sc.pp.normalize_total`, `sc.pp.log1p`, `sc.pp.highly_variable_genes`, `sc.tl.pca`, `sc.pp.neighbors`, `sc.tl.leiden`, `sc.tl.umap`.\n"
            "To compute mitochondrial QC metrics, annotate mt genes with `adata.var['mt'] = adata.var_names.str.upper().str.startswith('MT-')`\n"
            "and use `sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)`.\n"
            "Use `total_counts` (not `n_counts`)."
            ),
        "CellTypist": "Python tool for automatic cell type annotation using pretrained models.",
        "scVI": "Python deep generative model for batch correction and latent embedding.",
        "PAGA": "Scanpy graph abstraction for trajectory/clustering visualization."
    }

    tool_info = "\n".join(f"- {t}: {tool_docs.get(t, 'N/A')}" for t in py_tools)

    # Anchor correct CellTypist usage in few-shot guidelines
    celltypist_hint = (
        """
# CellTypist reference usage (guideline):
import celltypist
model = celltypist.models.download_model('Immune_All_Low.pkl')
pred = celltypist.annotate(adata, model=model, majority_voting=True, group_by='leiden')
adata.obs['celltype'] = pred.predicted_labels
        """.strip()
    )

    return f"""

You are an expert in scRNA-seq and Python.
Generate ONLY Python code (no R). Do not reload datasets. Use the existing variable `adata` (AnnData) and objects created by previous steps.
Avoid redundant imports; prefer using `sc` and `adata` already in scope.
FORBIDDEN: placeholders like 'path/to/your/data.h5ad'.
Avoid deprecated or invalid API calls like `sc.pp.pca` — use `sc.tl.pca` instead.
Clustering should be done using `sc.tl.leiden`, not `sc.tl.louvain`.


Step to implement: {step_description}

Available tools:
{', '.join(py_tools) if py_tools else '(none)'}

Tool notes:
{tool_info if tool_info else '(no details)'}

Previous context (you MAY reuse variables defined before):
{context_code}

Return ONLY JSON with fields "analysis" and "code":
{{
  "analysis": "Brief explanation of what the code does and expected side-effects on adata.",
  "code": "<executable python code that runs in this environment>"
}}

If the step is about annotation with CellTypist, follow this usage pattern (adapt to context):
{celltypist_hint}
"""


def make_evaluator_prompt(step_description: str, code_trials: List[str], user_goal: str) -> str:
    header = f"""
You are an expert in scRNA-seq. Evaluate multiple Python code candidates for the step: "{step_description}".
User goal: {user_goal}.

Criteria (in order):
1) Executability in Python with `adata` and `sc` in scope; no dataset reloads or undefined vars.
2) Alignment with the step.
3) Non-duplication and conciseness.
4) Expected side-effects on `adata` (e.g., create 'leiden', 'celltype', embeddings) consistent with Scanpy/CellTypist APIs.

Return ONLY JSON:
{{
  "selected_index": <index starting at 1>,
  "justification": "Short reason"
}}
"""
    parts = []
    for i, code in enumerate(code_trials, 1):
        parts.append(f"\n--- Code #{i} ---\n{code}")
    return header + "\n".join(parts)

In [5]:
# --- Core modules ------------------------------------------------------------

def planner_task_decomposition(user_task: str, dataset_description: str = "") -> List[Step]:
    prompt = make_planner_prompt(user_task, dataset_description)
    resp = client.chat.completions.create(
        model=MODEL,
        temperature=0.2,
        messages=[{"role": "user", "content": prompt}],
    )
    content = resp.choices[0].message.content
    data, err = extract_json(content)
    if err:
        print("[Planner] JSON parse error:", err)
        print("Raw planner content:\n", content)
        return []
    steps: List[Step] = []
    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict) and {"step", "title", "description"} <= set(item):
                steps.append(Step(step=int(item["step"]), title=str(item["title"]), description=str(item["description"])) )
    # sort by step index just in case
    steps.sort(key=lambda s: s.step)
    return steps


def executor_generate_code(step_description: str, available_tools: List[str], context_code: str = "") -> Tuple[str, str]:
    prompt = make_executor_prompt(step_description, available_tools, context_code)
    resp = client.chat.completions.create(
        model=MODEL,
        temperature=0.25,
        messages=[{"role": "user", "content": prompt}],
    )
    content = resp.choices[0].message.content
    parsed, err = extract_json(content)
    if err or not isinstance(parsed, dict):
        print("[Executor] JSON parse error:", err)
        print("Raw executor content:\n", content)
        return "", ""
    return parsed.get("code", ""), parsed.get("analysis", "")


def evaluator_select_best_code(step_description: str, code_trials: List[str], user_goal: str = "accurate result") -> Tuple[Optional[int], str]:
    prompt = make_evaluator_prompt(step_description, code_trials, user_goal)
    resp = client.chat.completions.create(
        model=MODEL,
        temperature=0.2,
        messages=[{"role": "user", "content": prompt}],
    )
    content = resp.choices[0].message.content
    parsed, err = extract_json(content)
    if err or not isinstance(parsed, dict):
        print("[Evaluator] JSON parse error:", err)
        print("Raw evaluator content:\n", content)
        return None, ""
    idx = parsed.get("selected_index")
    just = parsed.get("justification", "")
    if isinstance(idx, int):
        return idx, just
    try:
        return int(idx), just  # sometimes strings
    except Exception:
        return None, just

In [6]:
# --- Runner / Orchestrator ---------------------------------------------------

def run_cellagent_task(
    user_task: str,
    dataset_description: str,
    available_tools: List[str],
    steps_to_run: Optional[List[int]] = None,
    trials_per_step: int = 2,
    user_goal: str = "accurate result",
) -> List[Result]:

    print("\nTask:", user_task)
    print("Dataset:", dataset_description)
    print("Available tools:", ", ".join(available_tools) if available_tools else "(none)")

    plan = planner_task_decomposition(user_task, dataset_description)
    if not plan:
        print("No plan was generated.")
        return []

    results: List[Result] = []
    memory_code = ""  # accumulate chosen code for context

    for step in plan:
        if steps_to_run and step.step not in steps_to_run:
            continue

        print(f"\nStep {step.step}: {step.title}\n{step.description}")

        # Generate trials
        trials: List[str] = []
        for _ in range(max(1, trials_per_step)):
            code, _analysis = executor_generate_code(step.description, available_tools, context_code=memory_code)
            if code and code not in trials:
                trials.append(code)

        if not trials:
            print("No code was generated for this step.")
            continue

        # Evaluate
        idx, justification = evaluator_select_best_code(step.description, trials, user_goal=user_goal)
        if not idx or not (1 <= idx <= len(trials)):
            # fallback to first if evaluator failed
            idx = 1
            if not justification:
                justification = "Fallback to first candidate due to evaluation parse failure."

        best_code = trials[idx - 1]
        print("Selected code:\n", best_code)
        print("Motivation:", justification)

        results.append(Result(step=step.step, title=step.title, description=step.description, code=best_code, justification=justification))

        # memory for subsequent steps
        memory_code += f"\n# Step {step.step}: {step.title}\n" + best_code + "\n"

    return results

In [11]:
# --- Example usage (comment/uncomment as needed) ----------------------------
# Esempio di utilizzo semplice in Google Colab

# 1. Carico il dataset
filename = "pbmc.h5ad"
import scanpy as sc
adata = sc.read_h5ad(filename)
print(f"Dataset loaded: {adata.n_obs} cells, {adata.n_vars} genes.")

# 2. Configuro l'API OpenAI
from google.colab import userdata
from openai import OpenAI
api_key = userdata.get("API-KEY")
if not api_key:
    raise RuntimeError("API key mancante in Colab userdata")
client = OpenAI(api_key=api_key)

# 3. Definisco il task
user_task = "Annotate the cell types in the dataset."
dataset_description = "Human peripheral blood mononuclear cells (PBMC), single-cell RNA-seq."
available_tools = ["Scanpy", "CellTypist", "ScType"]

# 4. Eseguo la pipeline
results = run_cellagent_task(
    user_task=user_task,
    dataset_description=dataset_description,
    available_tools=available_tools,
    steps_to_run=None,       # Esegue tutti gli step
    trials_per_step=2,       # Due tentativi per step
    user_goal="accurate result"
)

# 5. Stampo i risultati
print("\n\nFinal selected code per step:")
for r in results:
    print(f"\nStep {r.step}: {r.title}")
    print(r.code)
    print("Justification:", r.justification)


Dataset loaded: 1087 cells, 15099 genes.

Task: Annotate the cell types in the dataset.
Dataset: Human peripheral blood mononuclear cells (PBMC), single-cell RNA-seq.
Available tools: Scanpy, CellTypist, ScType

Step 1: Load Data
Import the PBMC single-cell RNA-seq data into an AnnData object using Scanpy.
Selected code:
 adata.var['mt'] = adata.var_names.str.upper().str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)
Motivation: Code #1 only performs quality control metrics calculation, which aligns with the initial data import step without additional processing or side effects.

Step 2: QC & Normalization
Filter low-quality cells based on metrics like total counts and gene counts; normalize counts and apply log1p transformation.
Selected code:
 sc.pp.filter_cells(adata, min_counts=500)  # Filter cells with fewer than 500 total counts
sc.pp.filter_cells(adata, min_genes=200)  # Filter cells with fewer than 200 genes detected
sc.pp.normalize_total(adat